<h1 style='color: blue; font-size: 34px; font-weight: bold;'> Projeto Proposto 
</h1>
<p style='font-size: 18px; line-height: 2; margin: 0px 0px; text-align: justify; text-indent: 0px;'>    
<i> Este projeto tem o intuito de estudar Modelos de Séries Temporais para previsão da taxa de mortalidade em alguns países. </i>       
</p>  

# <font color='red' style='font-size: 40px;'> Problemática </font>
<hr style='border: 2px solid red;'>

<p style='font-size: 18px; line-height: 2; margin: 0px 0px; text-align: justify; text-indent: 0px;'>    
<i>  Este projeto tem o intuito de estudar Modelos de Séries Temporais para previsão da taxa de mortalidade em alguns países.


</i> 
</p>  

<p style='font-size: 18px; line-height: 2; margin: 0px 0px; text-align: justify; text-indent: 0px;'>    
<i> 
</i> 
</p>  


# <font color='red' style='font-size: 40px;'> Bibliotecas Utilizadas </font>
<hr style='border: 2px solid red;'>

In [1]:
## Bibliotecas De Manipulação de Dados e Visualização
import pandas as pd 
import geopandas as gpd
import builtins as builtins
from pyspark.sql.functions import * 
from pyspark.sql.types import *
from pyspark.sql.window import Window
from pyspark.sql import SparkSession, Row, functions as F 
import matplotlib.pyplot as plt
import seaborn as sns 
from IPython.display import display, Image

## Bibliotecas de Modelagem Matemática e Estatística
import numpy as np
import scipy as sp 
import scipy.stats as stats
import statsmodels
import statsmodels.api as sm
import statsmodels.formula.api as smf
from scipy.stats import normaltest, ttest_ind, ttest_rel, mannwhitneyu, wilcoxon, kruskal
from statsmodels.stats.weightstats import ztest
from statsmodels.stats.diagnostic import lilliefors

# Bibliotecas de Manipulação de Tempo
import time
import datetime

# Bibiliotecas de Seleção de Modelos
from sklearn.model_selection import train_test_split, KFold, cross_val_score, cross_validate, GridSearchCV
from skopt import BayesSearchCV
from skopt.space import Real, Categorical, Integer 
from sklearn.feature_selection import RFE, VarianceThreshold
from feature_engine.selection import DropConstantFeatures, DropCorrelatedFeatures, SmartCorrelatedSelection

# Bibliotecas de Pré-Processamento e Pipeline
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from category_encoders import TargetEncoder, BinaryEncoder
import category_encoders as ce 
from sklearn.pipeline import Pipeline, make_pipeline
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer, KNNImputer

# Bibliotecas de Modelos de Machine Learning
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor

# Bibliotecas de Métricas de Machine Learning
from sklearn.metrics import mean_absolute_error, mean_squared_error, mean_absolute_percentage_error

# Parâmetros de Otimização
import warnings
%matplotlib inline
sns.set()
plt.rcParams['font.family'] = 'Arial'
plt.rcParams['font.size'] = '14'
plt.rcParams['figure.figsize'] = [10, 5]
pd.set_option('display.max_rows', 100)
pd.set_option('display.max_columns', 100)
pd.set_option('display.float_format', lambda x: '%.2f' % x) # Tira os números do formato de Notação Científica
np.set_printoptions(suppress=True) # Tira os números do formato de Notação Científica em Numpy Arrays
warnings.filterwarnings('ignore')
warnings.simplefilter(action='ignore', category=FutureWarning) # Retira Future Warnings

# Spark Session
spark = SparkSession.builder.getOrCreate()

# <font color='red' style='font-size: 40px;'> Funções </font>
<hr style='border: 2px solid red;'>

In [2]:
def plota_temporal(df, titulo, x, y):
    plt.figure(figsize=(10, 6))
    plt.plot(df, color='#1FB3E5', linewidth=2)
    plt.scatter(df.index, df, color='#1FB3E5', s=40)
    plt.title(f"{titulo}", fontsize=14)  # Define o título do gráfico e o tamanho da fonte
    plt.xlabel(f"{x}", fontsize=12)  # Define o rótulo do eixo x e o tamanho da fonte
    plt.ylabel(f"{y}", fontsize=12)  # Define o rótulo do eixo y e o tamanho da fonte
    plt.xticks(df.index, fontsize=10)  # Define o tamanho da fonte dos valores no eixo x
    plt.yticks(fontsize=10)  # Define o tamanho da fonte dos valores no eixo y
    plt.grid(True, linestyle=':', alpha=0.5)  # Adiciona linhas de grade
    plt.tight_layout()  # Ajusta automaticamente a posição dos elementos do gráfico
    plt.show()  # Exibe o gráfico

In [3]:
def ano_mes_dia(df):

    df['timestamp'] = pd.to_datetime(df['timestamp'])
    df['year'] = df['timestamp'].dt.year
    df['month'] = df['timestamp'].dt.month
    df['day'] = df['timestamp'].dt.day


In [4]:
def separa_feature_target(target, dados):
    x = dados.drop(target, axis = 1)
    y = dados[[target]]
    return x, y

In [5]:
def separa_treino_teste_temporal(data, data_teste, dados):
    dados.sort_values(by = data, ascending = True, inplace = True)
    df_train = dados.loc[dados[data] < data_teste]
    df_test = dados.loc[dados[data] >= data_teste]

    return df_train, df_test

In [6]:
def discretiza_variavel(df, variavel_quant, variavel_qualit, bins, labels, right):
    df[variavel_qualit] = pd.cut(
        df[variavel_quant], 
        bins= bins, 
        labels= labels, 
        right = right
    )
    df.drop(variavel_quant, axis = 1, inplace = True)

In [7]:
def metricas(regressor, target, y_train, y_predict_train, y_test, y_predict_test, coeficiente_det_train, coeficiente_det_test):
    y_test[target].fillna(y_train[target].median(), inplace = True)

    mae_train = mean_absolute_error(y_predict_train, y_train)
    mse_train = mean_squared_error(y_predict_train, y_train)
    mape_train = mean_absolute_percentage_error(y_predict_train, y_train)
    metricas_treino = pd.DataFrame({'Coeficiente de Determinação (R^2)':coeficiente_det_train, 'Erro_Medio_Absoluto':mae_train, 'Erro_Quadratico_Medio':mse_train, 'Media_Percentual_Absoluta_Erro':mape_train, 'Etapa':'treino', 'Regressor':regressor}, index = np.arange(1, 2))


    mae_test = mean_absolute_error(y_predict_test, y_test)
    mse_test = mean_squared_error(y_predict_test, y_test)
    mape_test = mean_absolute_percentage_error(y_predict_test, y_test)
    metricas_teste = pd.DataFrame({'Coeficiente de Determinação (R^2)':coeficiente_det_test, 'Erro_Medio_Absoluto':mae_test, 'Erro_Quadratico_Medio':mse_test, 'Media_Percentual_Absoluta_Erro':mape_test, 'Etapa':'teste', 'Regressor':regressor}, index = np.arange(1, 2))

    metricas_finais = pd.concat([metricas_treino, metricas_teste])

    return metricas_finais

def metricas_modelos_juntos(lista_modelos):
    metricas_modelos = pd.concat(lista_modelos).set_index('Regressor')
    return metricas_modelos


# <font color='red' style='font-size: 40px;'> 1) Leitura do Dataset </font>
<hr style='border: 2px solid red;'>

# <font color='green' style='font-size: 30px;'> 1.1) Descrição das Variáveis </font>
<hr style='border: 2px solid green;'>

1. **Country Code:** Um código que representa o país.

2. **Country Name:** O nome do país.

3. **Year:** O ano dos dados.

4. **Age Group:** O grupo etário dos indivíduos.

5. **Sex:** O sexo dos indivíduos (Masculino, Feminino, Ambos).

6. **Number of Deaths:** O número de mortes.

7. **Death Rate Per 100,000:** A taxa de mortalidade por 100.000 indivíduos.

In [ ]:
df_raw = pd.read_csv('./data/IHME_GBD_2010_MORTALITY_AGE_SPECIFIC_BY_COUNTRY_1970_2010.csv')\

# df_cases['city_code'] = df_cases['code'].astype(int)
# df_cases['cases'] = df_cases['cases'].astype(int)
# df_raw_cities['city_code'] = df_raw_cities['city_code'].astype(str).str[:-1].astype(int)

# <font color='red' style='font-size: 40px;'> 2) Análise Exploratória </font>
<hr style='border: 2px solid red;'>

> 1. Para a análise, vamos explorar os seguintes tópicos:

- 1. Qual país tem a maior média de taxa de mortalidade por 100.000 indivíduos ao longo de todos os anos?
- 2. Qual é a tendência na taxa de mortalidade por 100.000 indivíduos ao longo dos anos para os países da América do Sul? Trace um gráfico de linhas contendo todos os países e a evolução das mortes ao longo dos anos.
- 3. Existem diferenças significativas nas taxas de mortalidade entre homens e mulheres em diferentes grupos etários para os países da América do Sul?​
- 4. Como a taxa de mortalidade por 100.000 indivíduos varia entre diferentes grupos etários dentro do Brasil? Utilize um histograma contendo a distribuição das mortes por ano para cada uma das faixas etárias.
- 5. Qual a evolução das mortes por gênero pra o Brasil? Trace um gráfico de linhas com cada uma delas representando um dos gêneros ao longo dos anos.​
- 6. Qual grupo etário tem o maior número de mortes no Brasil? Este número é o mesmo quando olhamos para todos os anos em comparação com quando consideramos apenas os últimos 10 anos?
- 7. Qual país apresentou a melhoria mais significativa nas taxas de mortalidade por 100.000 indivíduos ao longo do período analisado?

In [40]:
df_cases_states_time_series = (
    df_cases.groupby(['date', 'state']).agg({
        "cases": "sum", 
        "deaths": "sum", 
    })
    .reset_index()
)

df_cases_states_time_series['tax_deaths_per_cases'] = df_cases_states_time_series['deaths']/df_cases_states_time_series['cases']*100
df_cases_states_time_series['tax_deaths_per_cases'] = df_cases_states_time_series['tax_deaths_per_cases'].fillna(0)

df_cases_states_cross_section = (
    df_cases.groupby(['state']).agg({
        "cases": "sum", 
        "deaths": "sum", 
    })
    .reset_index()
)

df_cases_states_cross_section['tax_deaths_per_cases'] = df_cases_states_cross_section['deaths']/df_cases_states_cross_section['cases']*100
df_cases_states_cross_section['tax_deaths_per_cases'] = df_cases_states_cross_section['tax_deaths_per_cases'].fillna(0)


# <font color='green' style='font-size: 30px;'> 2.1) Distribuição Geral de Casos e Mortes </font>
<hr style='border: 2px solid green;'>

# <font color='green' style='font-size: 30px;'> 2.2) Distribuição Acumulada </font>
<hr style='border: 2px solid green;'>

In [4]:
# # Agrupa por data e calcula a soma acumulada
# df_grouped = df_cases_states.sort_values(by='date').groupby('date')[['cases', 'deaths']].sum().cumsum().reset_index()

# print(df_grouped.head())  # Exibe os primeiros valores acumulados


In [5]:
# # Criando o gráfico
# plt.figure(figsize=(12,6))
# plt.plot(df_grouped['date'], df_grouped['cases'], label='Casos Acumulados', color='blue', linestyle='-', marker='o')
# plt.plot(df_grouped['date'], df_grouped['deaths'], label='Mortes Acumuladas', color='red', linestyle='-', marker='s')

# # Configurações do gráfico
# plt.xlabel('Data')
# plt.ylabel('Quantidade Acumulada')
# plt.title('Evolução Acumulada de Casos e Mortes')
# plt.xticks(rotation=45)  # Rotaciona as datas para melhor visualização
# plt.legend()
# plt.grid()

# # Exibir gráfico
# plt.show()


# <font color='green' style='font-size: 30px;'> 2.3) Distribuição ao Longo do Tempo </font>
<hr style='border: 2px solid green;'>

In [ ]:
# df_raw_groupped = df_raw.groupby(['Date', 'Country/Region']).agg({
#     "Lat": "max", 
#     "Long": "max", 
#     "Confirmed": "sum", 
#     "Recovered": "sum", 
#     "Deaths": "sum"
# }).reset_index()

In [ ]:
# plt.figure(figsize=(16,8))
# ax = plt.subplot(1,2,1)
# ax.set_title('Casos Acumulados', fontsize=18, loc='left')
# plt.plot(df_brazil['Brazil'], label='Brazil')
# plt.plot(df_conf_w['Argentina'], label='Argentina')
# plt.plot(df_conf_w['Uruguay'], label='Uruguay')
# plt.xlabel("Dia")
# plt.ylabel("Casos")
# plt.legend();
# ax = plt.subplot(1,2,2)
# ax.set_title('Óbitos Acumulados', fontsize=18, loc='left')
# plt.plot(df_brazil['Brazil'], label='Brazil')
# plt.plot(df_deaths_w['Argentina'], label='Argentina')
# plt.plot(df_deaths_w['Uruguay'], label='Uruguay')
# plt.xlabel("Dia")
# plt.ylabel("Óbitos")
# plt.legend()